In [1]:
from pyspark.sql import SparkSession
Spark=SparkSession.builder.appName('Missing').getOrCreate()

In [2]:
# read dataset
tranning = Spark.read.csv('NameDetails.csv',header=True,inferSchema=True)

In [3]:
tranning.show()

+--------+---+---------+------+
|    Name|Age|Epirience|Salary|
+--------+---+---------+------+
| Avinash| 28|        4|100000|
|Saipriya| 26|        3| 91000|
|   Anuja| 25|        8| 71000|
| Prajkta| 29|        7| 88000|
|  Poonam| 27|        6| 59000|
|Priyanka| 22|        1| 33000|
+--------+---+---------+------+



In [5]:
tranning.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Epirience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
tranning.columns

['Name', 'Age', 'Epirience', 'Salary']

In [8]:
# create a group all my independant variables, so we use vectorAssembler
from pyspark.ml.feature import VectorAssembler
featureAssemblr = VectorAssembler(inputCols=['Age','Epirience'],outputCol='Independant Featurs')  # using independat column we create new column as independant featurs

In [10]:
output = featureAssemblr.transform(tranning)

In [11]:
output.show()

+--------+---+---------+------+-------------------+
|    Name|Age|Epirience|Salary|Independant Featurs|
+--------+---+---------+------+-------------------+
| Avinash| 28|        4|100000|         [28.0,4.0]|
|Saipriya| 26|        3| 91000|         [26.0,3.0]|
|   Anuja| 25|        8| 71000|         [25.0,8.0]|
| Prajkta| 29|        7| 88000|         [29.0,7.0]|
|  Poonam| 27|        6| 59000|         [27.0,6.0]|
|Priyanka| 22|        1| 33000|         [22.0,1.0]|
+--------+---+---------+------+-------------------+



in the above in 'independent Feature' we combaine 2 columns i.e Age and Expirience 

In [12]:
output.columns

['Name', 'Age', 'Epirience', 'Salary', 'Independant Featurs']

In [13]:
finalized_data=output.select('Independant Featurs','Salary')

In [14]:
finalized_data.show()

+-------------------+------+
|Independant Featurs|Salary|
+-------------------+------+
|         [28.0,4.0]|100000|
|         [26.0,3.0]| 91000|
|         [25.0,8.0]| 71000|
|         [29.0,7.0]| 88000|
|         [27.0,6.0]| 59000|
|         [22.0,1.0]| 33000|
+-------------------+------+



In [19]:
# train, test, split using function
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independant Featurs', labelCol='Salary')
regressor=regressor.fit(train_data)

In [21]:
### coefficient
regressor.coefficients

DenseVector([8904.1219, -1552.8674])

In [22]:
### intercept 
regressor.intercept

-151818.99641576895

In [27]:
### prediction
pred_results=regressor.evaluate(test_data)

IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.

In [28]:
pred_results.predictions.show()

NameError: name 'pred_results' is not defined